In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import cv2


import torch
import torch.nn as nn
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

### Dataset и Dataloader

In [936]:
from dataloader import LipReadingVideoDataset, LipReadingVideoDatasetZIP, collate_func

In [919]:
#data_path = os.path.normpath('F:/lip_reading_data/LRS3/lrs3_trainval')
data_path_zip = os.path.normpath('../data/lrs3_trainval.zip')

In [937]:
from torchvideotransforms import video_transforms

flip = video_transforms.RandomHorizontalFlip(p=0.5)
rotation = video_transforms.RandomRotation(degrees=15)

transform = video_transforms.Compose([flip, rotation])

In [938]:
#ds = LipReadingVideoDataset(data_path)
ds_zip = LipReadingVideoDatasetZIP(data_path_zip, transform=transform)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:811: UserWarning: The following variables cannot be serialized: ds_zip
  warnings.warn(message)


In [939]:
dl = torch.utils.data.DataLoader(ds_zip, batch_size=4, shuffle=True, collate_fn=collate_func)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:811: UserWarning: The following variables cannot be serialized: dl, ds_zip
  warnings.warn(message)


In [940]:
for i, (vframes_batch, list_of_sentences) in enumerate(dl):
    print(vframes_batch.shape)
    if i >=1:
        break

torch.Size([4, 5, 128, 3, 224, 224])
torch.Size([4, 5, 128, 3, 224, 224])


/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives w

### Показ видео

In [941]:
sample = ds_zip[1]
vframes, text, txt_path, video_path = sample.values()

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:811: UserWarning: The following variables cannot be serialized: dl, ds_zip
  warnings.warn(message)


In [771]:
# не работает
def video_show(vframes):
    time, C, H, W = vframes.shape
    for i in range(time):
        frame = vframes[i].numpy()
        
        cv2.imshow('frame', frame)
        # & 0xFF is required for a 64-bit system
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

### Подготовка словаря

In [ ]:
sentences = []

for dir_path, dir_names, file_names in os.walk(data_path):
    # перебрать файлы
    for file_name in file_names:
        if file_name.endswith('.txt'):
            txt_path = os.path.join(dir_path, file_name)
            txt_file = open(txt_path, 'r')
            
            _, text = txt_file.readline().split(':') # прочитанный текст
            text = text.strip()
            
            sentences.append(text)
            txt_file.close()

In [942]:
file = open('sentences.txt', 'r')
sentences = list(map(lambda line: line.strip(), file.readlines()))
file.close()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:811: UserWarning: The following variables cannot be serialized: file
  warnings.warn(message)


In [943]:
sentences[:5]

['ONE DAY A YOUNG BOY COMES UPON THE SUNFLOWER WHILE VISITING THE GARDEN AND HE NOTICES HOW WEAK IT LOOKS',
 'JUST LIKE REACHING OUT TO THE SUNFLOWER BY PROVIDING SOMEONE WHO IS NEGLECTED ISOLATED OR FORGOTTEN',
 'THEY ARE INCREDIBLE YET SO OFTEN THEY ARE FORGOTTEN LACKING THE LOVE AND APPRECIATION THEY DESERVE',
 'I WILL HAVE HOPEFULLY MET MY GOAL',
 'THAT WAS COMPLETELY WRONG']

In [944]:
from collections import Counter
import nltk

tokenizer = nltk.tokenize.WordPunctTokenizer()
#tokenizer = nltk.tokenize.TreebankWordTokenizer()

token_counts = Counter(tokenizer.tokenize(' '.join(sentences)))

In [945]:
min_count = 1

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = []
for key, values in token_counts.items():
    if values >= min_count:
        tokens.append(key)

# Add a special tokens for unknown and empty words
BOS, EOS, UNK, PAD = '_BOS_', '_EOS_', '_UNK_', '_PAD_'
tokens = [PAD, BOS, EOS, UNK] + tokens

In [946]:
len(tokens)

17193

In [947]:
# словари
token_to_id = {tok: i for i, tok in enumerate(tokens)}
id_to_token = {i: tok for tok, i in token_to_id.items()}

### Network

In [948]:
vframes_batch.shape

torch.Size([4, 5, 128, 3, 224, 224])

In [949]:
list_of_sentences

['AND THIS IS IMPORTANT BECAUSE THIS ENABLES US TO LOOK AT THE UNIVERSE IN A REGIME WHERE WE HAVE NEVER',
 'SO I SAT SARAH DOWN RIGHT IN FRONT OF ME',
 "OK THAT'S MY HOOK",
 'CASE IN POINT']

In [950]:
def sentences_to_tokens(list_of_sentences, tokenaizer):
    list_of_tokens = []
    for sentence in list_of_sentences:
        list_of_tokens.append(tokenizer.tokenize(sentence))
    return list_of_tokens

In [951]:
list_of_tokens = sentences_to_tokens(list_of_sentences, tokenizer)

In [952]:
list_of_tokens

[['AND',
  'THIS',
  'IS',
  'IMPORTANT',
  'BECAUSE',
  'THIS',
  'ENABLES',
  'US',
  'TO',
  'LOOK',
  'AT',
  'THE',
  'UNIVERSE',
  'IN',
  'A',
  'REGIME',
  'WHERE',
  'WE',
  'HAVE',
  'NEVER'],
 ['SO', 'I', 'SAT', 'SARAH', 'DOWN', 'RIGHT', 'IN', 'FRONT', 'OF', 'ME'],
 ['OK', 'THAT', "'", 'S', 'MY', 'HOOK'],
 ['CASE', 'IN', 'POINT']]

### Network

In [978]:
from network import LipReadinNN

In [979]:
my_network = LipReadinNN(token_to_id)

In [991]:
my_network.train(True)

with torch.no_grad():
    output = my_network(vframes_batch, list_of_tokens)

#output = my_network(vframes_batch, list_of_tokens)

In [992]:
out_sent = list(map(lambda l: ' '.join(l), output))

In [993]:
out_sent

['MEDICINAL HABITUATED DENIED CHOSE CATAPULTED INCONSISTENT PROMPTLY NICKY IMPLACABLY 271 CHOSE HABITUATED CATAPULTED SHOPPING CATAPULTED CHOSE BYPRODUCTS RE CONCEPTUALIZING CHOSE CHOSE CHOSE',
 'CHOSE CONCEPTUALIZING CHROMOSOMES DRUMMING CHOSE CHOSE BOLTZMANN FITS CHOSE MILLENNIA CHOSE LIFESPAN FITS BOUNDARY GASTON CHOSE CHOSE UTAH DRUMMING GREETER IMPARTING CHOSE',
 'EMBARRASSED CHOSE CHOSE CHOSE CHOSE HABITUATED CHOSE DISCUSSIONS INCONSISTENT READMITTED GET CHOSE READMITTED SO INCONSISTENT CHOSE INCONSISTENT TB DRUMMING CHOSE CHOSE FULBRIGHT',
 'INCONSISTENT INCONSISTENT DRUMMING GROWTH RECOMMENDATION RECOMMIT RECOMMIT IMPRESSIONISM INCONSISTENT SOY BOLTZMANN RECOMMENDATION CONCEPTUALIZING INCONSISTENT GROWTH FULBRIGHT CONCEPTUALIZING VULNERABLE SURVIVOR VISCOUS DRIVES BOLTZMANN']

In [1000]:
model = LipReadinNN(token_to_id)
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [1001]:
train_loss = train_model(model, dl, opt)

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives w

In [1002]:
train_loss

[9.55884838104248, 9.420223236083984]

In [1010]:
#!g1.1
torch.cuda.is_available()

Execute error: Servant g1.1 not allocated: Allocation was interrupted

In [1007]:
#!g1.1
torch.cuda.is_initialized()

False

In [997]:
#!g1.1
def train_model(model, dataloader, optimizer):
    model.train(True)

    train_loss = []

    for i, (vframes_batch, list_of_sentences) in enumerate(dataloader):
        list_of_tokens = sentences_to_tokens(list_of_sentences, tokenizer)
        if i >=2:
            break

        optimizer.zero_grad()
        output = model(vframes_batch, list_of_tokens)
        
        loss = model.loss
        train_loss.append(loss.item())
        loss.backward()
        
        optimizer.step()
    
    return train_loss

In [998]:
def val_model(model, dataloader):
    model.train(True)

    val_loss = []

    for i, (vframes_batch, list_of_sentences) in enumerate(dataloader):
        list_of_tokens = sentences_to_tokens(list_of_sentences, tokenizer)
        if i >=2:
            break
        with torch.no_grad(): # на val просто прогоняем модель, не собирая grad
            output = model(vframes_batch, list_of_tokens)

            loss = model.loss
            val_loss.append(loss.item())
    
    return val_loss

In [999]:
def test_model(model, dataloader):
    return val_model(model, dataloader) # то же самое, что val, но model никогда не видела data